In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl
from itertools import islice

run_id = 181
qid = 1281

In [2]:
from openpyxl import Workbook, load_workbook

In [3]:
wb = load_workbook('../data/bibliometric nodes.xlsx')
ws = wb['useme']
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx, columns=cols)

In [4]:
def get_topic(x):
    try:
        doi = x.replace('http://dx.doi.org/','')
        p = Doc.objects.get(wosarticle__di=doi)
        topic = DocTopic.objects.filter(run_id=run_id,doc=p).order_by('-score').first().topic.title
    except:
        topic = ""
    return(topic)

In [5]:
df['main_topic'] = df.apply(lambda x: get_topic(x['url']),axis=1)

In [7]:
df.to_csv('../data/bibliometric_nodes.csv',index=False)